In [ ]:
from glob import glob 
import os
from solucion_omar.paper2 import read_etsp_file, filter_arcs_by_time_window, hybrid_sa_ts

In [ ]:

pathsProblems = sorted(glob(os.path.join('Dataset', 'Problems', 'Problems', 'ETSPTW-MCR', '*', '*.txt')))
pathsSolution = sorted(glob(os.path.join('Dataset', 'Solutions', 'Solutions', 'ETSPTW-MCR', '*', '*.txt')))

In [ ]:
def get_solution_cost(path):
    with open(path) as file:
        lines = file.readlines()
        return int(lines[1])

178

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        problemPath = self.data[idx]
        optimalCost = get_solution_cost(self.labels[idx])

        data = read_etsp_file(problemPath)

        filtered_matrix, last_node = filter_arcs_by_time_window(data['nodes'], data['distance_matrix'])
        best_solution, best_cost, cost_progression, best_cost_progression = hybrid_sa_ts(
        nodes=data['nodes'],
        distance_matrix=data['distance_matrix'],
        num_customers=data['num_customers'],
        battery_capacity=data['battery_capacity'],
        energy_rate=data['energy_rate'],
        max_iterations=15000,     
        T0=10000,                 
        alpha=0.95,               
        cooling_interval=100,     
        perturbation_interval=500, 
        R=30,                     
        tabu_tenure_moves=400,     
        tabu_tenure_stations=75)

        gap = (best_cost - optimalCost)/optimalCost * 100

        return optimalCost, best_cost, gap
    
    

In [18]:
dataset = MyDataset(pathsProblems, pathsSolution)
dataloader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    num_workers=8,
    pin_memory=False,
    drop_last=False
)